In [5]:
import sys
import numpy as np
import random
import pygame
import math
import matplotlib.pyplot as plt
import imageio
import os
from sklearn.cluster import DBSCAN

base_path = "C:\\Users\\user\\Downloads\\"  # Ruta actualizada
os.makedirs(base_path, exist_ok=True)  # Esto creará el directorio si no existe

# Constantes y Parámetros de Simulación
screen_width, screen_height = 800, 800
n = int(input("Dime un número de cuerpos: "))  # Número de partículas ingresado por el usuario
G = 1  # Constante gravitacional, simplificada para la simulación
softening_factor = 0.1  # Factor de suavizado para prevenir singularidades y simular partículas sin colisiones

M = 1
ε = 0.0001
β = 1.0
δ = 10.0

# Constantes
G_0 = 6.67430e-11  # Constante gravitacional en m^3 kg^-1 s^-2
M_0 = 1.989e30   # Masa solar en kg
R_0 = 1.496e11   # Unidad astronómica en metros

T_0 = np.sqrt(R_0**3/(G_0*M_0))

c_0 = 3e8  # Velocidad de la luz en metros por segundo

c = c_0 * (T_0/R_0)

r_s = 2 * G * M / c**2

R_s = δ * r_s

R = ε**(-1/3) * n * δ * r_s

print(f"R_s es: {R_s}")

print(f"R es: {R}")

# Modificar los valores de dt y simulation_duration a conveniencia

T_exe = (2 * np.sqrt(2) * np.pi * R**(3/2) / (np.sqrt(G * M * δ * n)))
dt = T_exe / 1000
simulation_duration = 100 * T_exe

print(f"T_exe es: {T_exe}")
print(f"dt es: {dt}")
print(f"La duración de la simulación será: {simulation_duration}")

# Inicializar pygame para visualización
pygame.init()
screen = pygame.display.set_mode((screen_width, screen_height))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 24)

# Preguntar al usuario si quiere correcciones relativistas y pérdida de masa
use_relativistic_corrections = input("¿Quieres usar correcciones relativistas? (yes/no): ").lower() == "yes"
use_mass_loss = input("¿Quieres considerar la pérdida de masa por emisión de ondas gravitacionales? (yes/no): ").lower() == "yes"

def initialize_particles(n):
    masses = [δ for _ in range(n)]
    
    positions = []
    velocities = []
    for _ in range(n):
        # Posiciones en coordenadas esféricas
        r = R * (np.random.uniform(0, 1))**(1/3)  # Corregido para distribución uniforme
        θ = np.random.uniform(0, np.pi)  # Ángulo polar
        Φ = np.random.uniform(0, 2 * np.pi)  # Ángulo azimutal
        
        # Convertir coordenadas esféricas a cartesianas
        x = r * np.sin(θ) * np.cos(Φ)
        y = r * np.sin(θ) * np.sin(Φ)
        z = r * np.cos(θ)
        positions.append(np.array([x, y, z]))
        
        # Velocidades
        # Calcular la velocidad usando la aproximación del teorema del virial
        potential_energy = -G * M / r
        kinetic_energy = -0.5 * potential_energy
        mean_v = np.sqrt(2 * kinetic_energy / M)
        
        v = np.random.normal(mean_v, mean_v / 100)
        
        # Dirección de la velocidad
        θ_v = np.random.uniform(0, np.pi)
        Φ_v = np.random.uniform(0, 2 * np.pi)
        vx = v * np.sin(θ_v) * np.cos(Φ_v)
        vy = v * np.sin(θ_v) * np.sin(Φ_v)
        vz = v * np.cos(θ_v)
        velocities.append(np.array([vx, vy, vz]))
        
    accelerations = [np.zeros(3) for _ in range(n)]
    half_step_velocities = [v + 0.5 * a * dt for v, a in zip(velocities, accelerations)]
    return masses, positions, velocities, accelerations, half_step_velocities

def cluster_positions(positions, eps=0.5, min_samples=2):
    """Agrupa posiciones usando DBSCAN y retorna etiquetas de clúster."""
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(positions)
    return clustering.labels_

# Generar una gran cantidad de colores distintos
def generate_color_pool(num_colors=50):
    np.random.seed(42)  # Opcional: para colores reproducibles
    return [tuple(np.random.choice(range(256), size=3)) for _ in range(num_colors)]

color_pool = generate_color_pool(50)  # Generar 50 colores distintos
color_map = {}  # Inicializar un mapa de colores vacío

# Función para actualizar el mapa de colores con nuevas etiquetas de clúster
def update_color_map(labels, color_map, color_pool):
    existing_labels = set(color_map.keys())
    new_labels = set(labels) - existing_labels
    
    for label in new_labels:
        if label not in color_map:
            color_map[label] = color_pool[len(color_map) % len(color_pool)]

def calculate_accelerations(masses, positions, velocities):
    """Calcula las aceleraciones gravitacionales actuando en cada partícula basado en la gravedad Newtoniana y correcciones relativistas añadidas."""
    n = len(masses)  # Número de partículas
    accelerations = [np.zeros(3) for _ in range(n)]
    collisions = [] 

    for i in range(n):
        for j in range(i + 1, n):
            r_ij = positions[j] - positions[i]
            distance = np.linalg.norm(r_ij)
            distance_softened = np.sqrt(distance**2 + softening_factor**2)
            unit_vector = r_ij / distance
            unit_vector_2 = -r_ij / distance
            v_i = velocities[i]
            v_j = velocities[j] 
            v_ij = velocities[j] - velocities[i]  # Vector de velocidad relativa
            
            Rs_i = 2 * G * masses[i] / c**2
            Rs_j = 2 * G * masses[j] / c**2
            
            # Verificar colisiones basadas en los radios de Schwarzschild
            if distance < Rs_i:
                collisions.append(j)  # j cae en i
            elif distance < Rs_j:
                collisions.append(i)  # i cae en j
            
            # Término de gravedad newtoniana
            newtonian_term = G * masses[j] / (distance_softened**2) * unit_vector
            newtonian_term_2 = G * masses[i] / (distance_softened**2) * unit_vector_2

            if use_relativistic_corrections:
                # Términos adicionales de corrección relativista
                relativistic_correction_1 = (2 * Rs_j / distance_softened + 2.5 * Rs_i / distance_softened) - (np.linalg.norm(v_i)**2 + 4 * np.dot(v_i, v_j) - 2 * np.linalg.norm(v_j)**2 + 1.5 * (np.dot(v_j, unit_vector))**2) / c**2
                
                for k in range(j + 1, n):
                    Rs_k = 2 * G * masses[k] / c**2
                    r_jk_v = positions[k] - positions[j]
                    r_ik_v = positions[k] - positions[i]
                    r_jk = np.sqrt(np.linalg.norm(r_jk_v)**2 + softening_factor**2)
                    r_ik = np.sqrt(np.linalg.norm(r_ik_v)**2 + softening_factor**2)
                    
                    new_term_1 = Rs_k / (2 * r_jk) + 2 * Rs_k / r_ik - Rs_k / (4 * r_jk**3) * np.dot(r_ij, r_jk_v)
                    relativistic_correction_1 += new_term_1
                    
                relativistic_correction_2 = -(Rs_j / (2 * distance_softened**2)) * np.dot((4 * velocities[i] - 3 * velocities[j]), v_ij) * unit_vector
                
                relativistic_correction_22 = -7 / 4 * Rs_j / distance
                
                relativistic_correction_222 = 0
                
                for k in range(j + 1, n):
                    r_jk_v = positions[k] - positions[j]
                    r_ik_v = positions[k] - positions[i]
                    r_jk = np.sqrt(np.linalg.norm(r_jk_v)**2 + softening_factor**2)
                    r_ik = np.sqrt(np.linalg.norm(r_ik_v)**2 + softening_factor**2)
                    
                    new_term_3 = G * masses[k] * r_jk_v / r_jk**3
                    relativistic_correction_222 += new_term_3
                    
                relativistic_correction_22 = relativistic_correction_22 * relativistic_correction_222
                relativistic_correction_2 = relativistic_correction_2 + relativistic_correction_22
                
                relativistic_correction_1_2 = (2 * Rs_i / distance_softened + 2 * Rs_j / distance_softened) - (np.linalg.norm(v_j)**2 + 4 * np.dot(v_j, v_i) - 2 * np.linalg.norm(v_i)**2 + 1.5 * (np.dot(v_i, unit_vector_2))**2) / c**2
                
                for k in range(j + 1, n):
                    Rs_k = 2 * G * masses[k] / c**2
                    r_jk_v = positions[k] - positions[j]
                    r_ik_v = positions[k] - positions[i]
                    r_jk = np.sqrt(np.linalg.norm(r_jk_v)**2 + softening_factor**2)
                    r_ik = np.sqrt(np.linalg.norm(r_ik_v)**2 + softening_factor**2)
                    
                    new_term_2 = Rs_k / (2 * r_jk) + 2 * Rs_k / r_jk - Rs_k / (4 * r_ik**3) * np.dot(-r_ij, r_ik_v)
                    relativistic_correction_1_2 += new_term_2
                    
                relativistic_correction_2_2 = -(Rs_i / (2 * distance_softened**2)) * np.dot((4 * velocities[j] - 3 * velocities[i]), -v_ij) * unit_vector_2
                
                relativistic_correction_2_22 = -7 / 4 * Rs_i / distance
                
                relativistic_correction_2_222 = 0
                
                for k in range(j + 1, n):
                    r_jk_v = positions[k] - positions[j]
                    r_ik_v = positions[k] - positions[i]
                    r_jk = np.sqrt(np.linalg.norm(r_jk_v)**2 + softening_factor**2)
                    r_ik = np.sqrt(np.linalg.norm(r_ik_v)**2 + softening_factor**2)
                    
                    new_term_4 = G * masses[k] * r_ik_v / r_ik**3
                    relativistic_correction_2_222 += new_term_4
                    
                relativistic_correction_2_22 = relativistic_correction_2_22 * relativistic_correction_2_222
                relativistic_correction_2_2 = relativistic_correction_2_2 + relativistic_correction_2_22
                
                # Combinar términos para el cálculo de la aceleración
                accelerations[i] += newtonian_term * (1 + relativistic_correction_1) + relativistic_correction_2
                accelerations[j] += (newtonian_term_2 * (1 + relativistic_correction_1_2) + relativistic_correction_2_2)
            else:
                accelerations[i] += newtonian_term
                accelerations[j] += newtonian_term_2
            
            collisions = list(set(collisions))
            
    return accelerations, collisions

def update_positions_velocities(masses, positions, velocities, accelerations, n):
    # Actualizar velocidad por medio paso
    half_step_velocities = [v + 0.5 * a * dt for v, a in zip(velocities, accelerations)]
    
    # Actualizar posición por un paso completo usando las velocidades de medio paso
    positions = [p + v_half * dt for p, v_half in zip(positions, half_step_velocities)]
    
    # Calcular nuevas aceleraciones a partir de las posiciones actualizadas
    new_accelerations, collisions = calculate_accelerations(masses, positions, velocities)
    
    # Completar el paso completo para las velocidades usando las nuevas aceleraciones
    velocities = [v_half + 0.5 * a_new * dt for v_half, a_new in zip(half_step_velocities, new_accelerations)]
    
    # Actualizar las aceleraciones para la próxima iteración
    accelerations = new_accelerations

    # Manejar colisiones basadas en el radio de Schwarzschild
    for index in sorted(collisions, reverse=True):
        Rs_current = 2 * G * masses[index] / c**2
        for other_index in range(len(masses)):
            if other_index != index:
                Rs_other = 2 * G * masses[other_index] / c**2
                if np.linalg.norm(positions[other_index] - positions[index]) < max(Rs_current, Rs_other):
                    if masses[index] >= masses[other_index]:  # La partícula actual absorbe a la otra
                        total_mass = masses[index] + masses[other_index]
                        final_mass = 0.9604 * total_mass if use_mass_loss else total_mass
                        velocities[index] = (velocities[index] + velocities[other_index]) / 2
                        print(f"{masses[other_index]} fusionada con {masses[index]} dando una masa de: {final_mass}")
                        masses[index] = final_mass
                        del masses[other_index]
                        del positions[other_index]
                        del velocities[other_index]
                        del accelerations[other_index]
                    else:  # La otra partícula absorbe a la actual
                        total_mass = masses[index] + masses[other_index]
                        final_mass = 0.9604 * total_mass if use_mass_loss else total_mass
                        velocities[other_index] = (velocities[index] + velocities[other_index]) / 2
                        print(f"{masses[index]} fusionada con {masses[other_index]} dando una masa de: {final_mass}")
                        masses[other_index] = final_mass
                        del masses[index]
                        del positions[index]
                        del velocities[index]
                        del accelerations[index]
                    n -= 1  # Reducir el conteo de partículas
                    break  # Salir del bucle después de manejar la colisión

    return positions, velocities, accelerations, masses, n

def calculate_densities(positions, n_shells=10, R_max=R):
    """
    Calcular densidades en cáscaras esféricas.
    n_shells: Número de cáscaras para dividir la esfera.
    R_max: Radio máximo de la esfera.
    Retorna densidades para cada cáscara.
    """
    shell_edges = np.linspace(0, R_max, n_shells+1)
    shell_volumes = 4/3 * np.pi * (shell_edges[1:]**3 - shell_edges[:-1]**3)
    shell_counts = np.zeros(n_shells)
    
    for pos in positions:
        r = np.linalg.norm(pos)
        if r < R_max:
            shell_index = int(n_shells * r / R_max)
            shell_counts[shell_index] += 1
            
    densities = shell_counts / shell_volumes
    return densities


#---------------------------------------------------------------------------------------------------------------#

# Inicializar partículas
masses, positions, velocities, accelerations, half_step_velocities = initialize_particles(n)

initial_conditions_path = os.path.join(base_path, "initial_conditions.npz")
np.savez(initial_conditions_path, masses=masses, posiciones=positions, velocities=velocities, accelerations=accelerations)
print(f"Condiciones iniciales guardadas en {initial_conditions_path}")

simulation_time = 0

time_history = []
positions_history = []
masses_history = []
color_history = []

# Bucle de simulación
running = True
while running and simulation_time <= simulation_duration:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    positions, velocities, accelerations, masses, n = update_positions_velocities(masses, positions, velocities, accelerations, n)
    
    cluster_labels = cluster_positions(np.array(positions)[:, :2])  # Agrupación basada en posiciones X, Y
    update_color_map(cluster_labels, color_map, color_pool)
    
    timestep_colors = [color_map[label] for label in cluster_labels]
    color_history.append(timestep_colors)
    
    screen.fill((0, 0, 0))  
    
    circle_radius = screen_width / 2  
    pygame.draw.circle(screen, (255, 255, 255), (screen_width // 2, screen_height // 2), int(circle_radius), 1)  # Borde de 1 píxel de ancho
    
    max_radius = R 
    scale_factor = screen_width / (2 * max_radius)  # Factor de escala para mapear el espacio de simulación al espacio de la pantalla
    for pos, label in zip(positions, cluster_labels):
        x_screen = (pos[0] + max_radius) * scale_factor
        y_screen = (pos[1] + max_radius) * scale_factor
        color = color_map.get(label, (255, 255, 255))  # El color por defecto es blanco
        pygame.draw.circle(screen, color, (int(x_screen), int(y_screen)), 5)
    
    pygame.display.flip()
    clock.tick(60)
    
    time_history.append(simulation_time)
    positions_history.append(np.array(positions))  # Hacer una copia de las posiciones actuales
    masses_history.append(np.array(masses))  # Registrar masas
    
    simulation_time += dt
    
final_conditions_path = os.path.join(base_path, "final_time_positions.npz")
np.savez(final_conditions_path, time=simulation_time, positions=positions)
print(f"Tiempo final y posiciones guardadas en {final_conditions_path}")

history_path = os.path.join(base_path, "simulation_history.npz")
np.savez(history_path, time=np.array(time_history), positions=np.array(positions_history), masas=np.array(masses_history), colors=np.array(color_history))
print(f"Historial de la simulación guardado en {history_path}")
    
pygame.quit()


Dime un número de cuerpos: 100
R_s es: 1.9719522727272723e-07
R es: 0.00042484423834508484
T_exe es: 2.460591193962022e-06
dt es: 2.4605911939620216e-09
La duración de la simulación será: 0.0002460591193962022
¿Quieres usar correcciones relativistas? (yes/no): no
¿Quieres considerar la pérdida de masa por emisión de ondas gravitacionales? (yes/no): no
Condiciones iniciales guardadas en C:\Users\Pablo\Downloads\initial_conditions.npz
Tiempo final y posiciones guardadas en C:\Users\Pablo\Downloads\final_time_positions.npz
Historial de la simulación guardado en C:\Users\Pablo\Downloads\simulation_history.npz
